In [ ]:
#Autor: Alonso Melgarejo
#Contacto: alonsoraulmgs@gmail.com
#Copyright: Big Data Academy

# Cargar imágenes como tensores

In [1]:
!pip install tf-nightly
#Nuestro dataset de imágenes está dividido en datos de entrenamiento y de validación
#Ambos datasets tenemos que convertirlos a tensores
#Trabajaremos primero con el dataset de entrenamiento, luego generalizaremos los códigos en una función

     |████████████████████████████████| 488.6 MB 25 kB/s 
     |████████████████████████████████| 462 kB 56.4 MB/s 
     |████████████████████████████████| 5.8 MB 11.1 MB/s 
     |████████████████████████████████| 1.3 MB 39.5 MB/s 
     |████████████████████████████████| 13.4 MB 236 kB/s 
     |████████████████████████████████| 2.1 MB 52.2 MB/s 


In [2]:
#Nos conectamos a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
#Deberemos indicarle la carpeta que queremos tensorizar
tensor = image_dataset_from_directory(
    '/content/drive/MyDrive/Data/IMAGES_RAW_SPLIT/train', #Ruta de la carpeta con imágenes
    color_mode="rgb", #Tipo de canal de color
    image_size=(32, 32), #Tamaño de las imágenes
    interpolation="bilinear", #El dataset puede tener imágenes que no sean de 32x32, en ese caso se redimenzionaran
    batch_size=10, #Es el parámetro que nos evita el colapso de la memoria RAM, procesaremos las imágenes de 10 en 10
    labels="inferred", #La carpeta divide las imágenes en subcategorías, al activar esta opción le estamos diciendo que cada subcarpeta es una categoría
    label_mode="int", #Cada categoría es etiquetada con un número (0, 1, 2)
    shuffle=True #Finalmente, al momento de tensorizar las imágenes, aleatoriezamos su orden para leerlas en desorden
)

Found 82 files belonging to 3 classes.


In [ ]:
#Si consultamos el objeto "class_names" del tensor, podemos encontrar el orden de las etiquetas que reconoció
#Se lee en el orden del array
#En este caso AVE es 0, GATO es 1 y perro es 2
tensor.class_names

['AVE', 'GATO', 'PERRO']

In [ ]:
#Veamos el tensor
tensor

#Notamos que aparece:
#(None, 32, 32, 3)
# "None" es la cantidad de imágenes tensorizadas, la librería no coloca nada aquí, deberemos ver el output anterior (82 imágenes)
# 32,32: Cada imagen está conformada por 32x32 píxeles
# 3: Como le indicamos un tipo de canal RGB, extrajo 3 canales de color

<BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
#Entonces, la dimensión del tensor de entrenamiento es:
# [82, 32, 32, 3]

In [ ]:
#IMPORTANTE: El tensor aún no existe, deberemos guardarlo en disco duro para que se cree

In [ ]:
#Hemos definido que leeremos las imágenes de 10 en 10 (batch_size=10)
#Como tenemos 82 imágenes, significa que se agruparán en 9 grupos de 10 imágenes cada uno, con excepción del último que tendrá 2 imágenes
#Para el procesamiento, cada grupo se cargará en memoria RAM, se convertirá en un tensor, se almacenará en el disco duro y se liberará la RAM para que entre el siguiente grupo

In [ ]:
#Iteraremos cada grupo
#Primero para analizar que hay dentro, vamos a mostrar el contenido del primer grupo de 10 imágenes
for tensor_features, tensor_labels in iter(tensor):
  tensor_features_1 = tensor_features
  tensor_labels_1 = tensor_labels

  #Con esto salimos del bucle en la primera iteración
  break

In [ ]:
#Si imprimos la dimensión del primer tensor para los features, nos encontramos con un tensor que contiene las 10 imágenes de 32x32 píxeles con 3 canales de color
tensor_features_1.shape

TensorShape([10, 32, 32, 3])

In [ ]:
#Si imprimos la dimensión del primer tensor para los labels, nos encontramos con un tensor de 10 labels
tensor_labels_1.shape

TensorShape([10])

In [ ]:
#Al ver su contenido notamos que para cada imagen cargada tensorialmente hay un label que indica si es AVE (0), GATO (1) o perro (2)
tensor_labels_1

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 2, 1, 1, 2, 2, 1, 1], dtype=int32)>

# Estandarizando el tipo de dato de los tensores de features y labels

In [ ]:
#Estas son las dimensiones del tensor de features
tensor_features_1.shape

TensorShape([10, 32, 32, 3])

In [ ]:
#Pero si consultamos el tipo de dato del tensor de features, veremos que es un "tensorflow.python.framework.ops.EagerTensor"
type(tensor_features_1)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
#Para trabajar de la manera más estándar posible, guardaremos la información en variables Numpy que cualquier otro Framework diferente a Tensorflow o Keras pueda procesarlo
#Convertimos el tensor tensorflow a un tensor de numpy
tensor_features_numpy_1 = tensor_features_1.numpy()

In [ ]:
#Si consultamos las dimensiones, sigue siendo el mismo tensor
tensor_features_numpy_1.shape

(10, 32, 32, 3)

In [ ]:
#Sólo ha variado el tipo de objeto que lo contiene
type(tensor_features_numpy_1)

numpy.ndarray

In [ ]:
#En el caso del tensor de labels tenemos que se indica si es AVE (0), GATO (1) o perro (2)
tensor_labels_1

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 2, 1, 1, 2, 2, 1, 1], dtype=int32)>

In [ ]:
#Ya sabemos que una red neuronal no puede procesar esto, debemos de crear un tensor de features con tres labels para AVE, GATO y PERRO
#Se colocará el valor de 1 en la clase correspondiente para cada imagen, en el resto 0

In [ ]:
#Importamos la función que categoriza
from tensorflow.keras.utils import to_categorical

In [ ]:
#Categorizamos el tensor
tensor_labels_categorico_1 = to_categorical(tensor_labels_1, num_classes = 3)
tensor_labels_categorico_1

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
#Si consultamos las dimensiones, tenemos un tensor de features de 10x3
tensor_labels_categorico_1.shape

(10, 3)

In [ ]:
#Este tensor también es de Numpy
type(tensor_labels_categorico_1)

numpy.ndarray

# Creando la variable que contiene la información de los tensores y se almacenará en disco duro

In [ ]:
#Vamos a crear una variable que tenga tres subcampos
# El primer campo (features) almacenará el tensor de features
# El segundo campo (labels) almacenará el tensor de labels
# El tercer campo (metadata) almacenará la descripción de los labels (LABEL 0: AVE, LABEL 1: GATO, LABEL 2: PERRO)

In [ ]:
#Creamos una variable de python que nos permita crear subcampos
dataset = {}

In [ ]:
#Creamos el primer campo
dataset['features'] = tensor_features_numpy_1
dataset['features'].shape

(10, 32, 32, 3)

In [ ]:
#Creamos el segundo campo
dataset['labels'] = tensor_labels_categorico_1
dataset['labels'].shape

(10, 3)

In [ ]:
#Creamos el tercer campo
dataset['metadata'] = tensor.class_names
dataset['metadata']

['AVE', 'GATO', 'PERRO']

# Guardar el tensor de imágenes en un archivo de disco duro

In [ ]:
#Finalmente, vamos a guardar el tensor en disco duro
#Los archivos de tensores los guardaremos en la siguiente ruta
ruta_tensores = '/content/drive/MyDrive/Data/TENSORS'

In [ ]:
#Borramos la carpeta por si ya existe

#Importamos la librería para manipular el sistema de archivos
import os

#Importamos la librería de borrado
import shutil

#Si existe, la borramos
if os.path.exists(ruta_tensores):
  shutil.rmtree(ruta_tensores)

In [ ]:
#Creamos la carpeta
os.mkdir(ruta_tensores)

In [ ]:
#Ya que los tensores están en arrays numpy, la variable "dataset" la guardaremos en un archivo comprimido para ahorrar espacio
#La extensión del archivo es "npz" y significa "Numpy ZIP"

In [ ]:
#Importamos la librería numpy
import numpy as np

In [ ]:
#Colocamos la variable "dataset" en un objeto numpy
dataset_numpy = np.array(dataset)

In [ ]:
#Guardamos la variable en la ruta elegida
np.savez(ruta_tensores+'/tensor_1', dataset_numpy)

In [ ]:
#Si entramos a la carpeta, veremos un archivo
#Recordemos que estamos agrupando las 82 imágenes en grupos de 10 en 10, hemos guardado el primer grupo, deberemos continuar con los siguientes

# Función utilitaria para almacenamiento de tensores en disco duro

In [ ]:
def almacenar_en_disco_como_tensores(tensor, ruta):
  #Si existe, la ruta, la borramos
  if os.path.exists(ruta):
    shutil.rmtree(ruta)

  #Creamos la carpeta en donde guardaremos los archivos de tensores
  os.mkdir(ruta)
  
  #Iteramos los tensores leídos
  index = 0
  for tensor_features, tensor_labels in iter(tensor):
    #Convertimos los features a array
    tensor_features_numpy = tensor_features.numpy()

    #Convertimos los labels a categóricos
    tensor_labels_categorico = to_categorical(tensor_labels, num_classes = len(tensor.class_names))

    #Colocamos los datos en un diccionario
    dataset = {}
    dataset['features'] = tensor_features_numpy
    dataset['labels'] = tensor_labels_categorico
    dataset['metadata'] = tensor.class_names

    #Guardamos los datos tensoriales en un archivo
    dataset_numpy = np.array(dataset)
    np.savez(ruta+'/tensor_' + str(index), dataset_numpy)

    index = index + 1

# Uso de la función para tensorizar los datos de entrenamiento

In [ ]:
#Leemos la carpeta de imágenes de entrenamiento como tensores
tensor_entrenamiento = image_dataset_from_directory(
    '/content/drive/MyDrive/Data/IMAGES_RAW_SPLIT/train', #Ruta de la carpeta con imágenes
    color_mode="rgb", #Tipo de canal de color
    image_size=(32, 32), #Tamaño de las imágenes
    interpolation="bilinear", #El dataset puede tener imágenes que no sean de 32x32, en ese caso se redimenzionaran
    batch_size=10, #Es el parámetro que nos evita el colapso de la memoria RAM, procesaremos las imágenes de 10 en 10
    labels="inferred", #La carpeta divide las imágenes en subcategorías, al activar esta opción le estamos diciendo que cada subcarpeta es una categoría
    label_mode="int", #Cada categoría es etiquetada con un número (0, 1, 2)
    shuffle=True #Finalmente, al momento de tensorizar las imágenes, aleatoriezamos su orden para leerlas en desorden
)

Found 82 files belonging to 3 classes.


In [ ]:
#Definimos la carpeta en donde almacenaremos los tensores
ruta_tensor_entrenamiento = '/content/drive/MyDrive/Data/TENSORS/train'

In [ ]:
#Creamos la carpeta en donde guardaremos los archivos de tensores
os.mkdir(ruta_tensor_entrenamiento)

In [ ]:
#Guardamos el tensor en la carpeta
almacenar_en_disco_como_tensores(tensor_entrenamiento, ruta_tensor_entrenamiento)

# Uso de la función para tensorizar los datos de validación

In [ ]:
#Leemos la carpeta de imágenes de validación como tensores
tensor_validacion = image_dataset_from_directory(
    '/content/drive/MyDrive/Data/IMAGES_RAW_SPLIT/test', #Ruta de la carpeta con imágenes
    color_mode="rgb", #Tipo de canal de color
    image_size=(32, 32), #Tamaño de las imágenes
    interpolation="bilinear", #El dataset puede tener imágenes que no sean de 32x32, en ese caso se redimenzionaran
    batch_size=10, #Es el parámetro que nos evita el colapso de la memoria RAM, procesaremos las imágenes de 10 en 10
    labels="inferred", #La carpeta divide las imágenes en subcategorías, al activar esta opción le estamos diciendo que cada subcarpeta es una categoría
    label_mode="int", #Cada categoría es etiquetada con un número (0, 1, 2)
    shuffle=True #Finalmente, al momento de tensorizar las imágenes, aleatoriezamos su orden para leerlas en desorden
)

Found 18 files belonging to 3 classes.


In [ ]:
#Definimos la carpeta en donde almacenaremos los tensores
ruta_tensor_validacion = '/content/drive/MyDrive/Data/TENSORS/test'

In [ ]:
#Creamos la carpeta en donde guardaremos los archivos de tensores
os.mkdir(ruta_tensor_validacion)

In [ ]:
#Guardamos el tensor en la carpeta
almacenar_en_disco_como_tensores(tensor_validacion, ruta_tensor_validacion)